In [0]:
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from datetime import datetime
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1w3nf2dE-fXLqReeIe3El99GgzsGr5Ecw'}) 
downloaded.GetContentFile('reddit_train.csv')    

reddit_train = pd.read_csv('reddit_train.csv')

downloaded = drive.CreateFile({'id':'1N35C1j6ufoUMKAaEC41c9NMOsKRNSsAX'})
downloaded.GetContentFile('reddit_test.csv')    

reddit_test = pd.read_csv('reddit_test.csv')

In [6]:
import nltk
nltk.download('popular')
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
import re

def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        elif tag.startswith('R'):
            yield wnl.lemmatize(word, pos='r')
            
        else:
            yield word


def msgProcessing(raw_msg):
    m_w=[]
    words2=[]
    raw_msg=str(raw_msg)
    raw_msg = str(raw_msg.lower())
    #url_stripper= re.sub(r'Email me.*[A-Z]',"",s)
    
    #raw_msg=re.sub(r'\w*[0-9]\w*','', url_stripper)
    raw_msg=re.sub(r'[^a-zA-Z]', ' ', raw_msg)
    
    words=raw_msg.lower().split()
    #Remove words with length lesser than 3 if not w in stops
    for i in words:
        if len(i)>=1:
            words2.append(i)
    stops=set(stopwords.words('english'))
    m_w=" ".join([w for w in words2])
    return(" ".join(lemmatize_all(m_w)))


def helperFunction(df):
    print ("Data Preprocessing!!!")
    
    num_msg=df.shape[0]
    clean_msg=[]
    for i in range(0,num_msg):
        clean_msg.append(msgProcessing(df[i]))
    X=clean_msg
    print ("Data Preprocessing Ends!!!")
    return X

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [0]:
# apply cross validation
def val6v1(rawData):

    test_dataset=rawData[60000:]
    train_dataset=rawData[:60000]
    
    return train_dataset,test_dataset

In [8]:
#vald
redditWords = helperFunction(reddit_train.T.iloc[1].to_numpy())
trainData, testData = val6v1(redditWords)
trainTar, testTar = val6v1(reddit_train.T.iloc[-1].to_numpy())
print(trainData[5])
print(testData[5])
print(trainTar)

Data Preprocessing!!!
Data Preprocessing Ends!!!
the joke be on you i ve only see it twice
never cease to amaze me how gullible you people be
['hockey' 'nba' 'leagueoflegends' ... 'nfl' 'nfl' 'nfl']


In [0]:
#realTest
# trainData = helperFunction(reddit_train.T.iloc[1].to_numpy())
# testData = helperFunction(reddit_test.T.iloc[1].to_numpy())
# trainTar = reddit_train.T.iloc[-1].to_numpy()
# print(trainData[5])
# print(testData[8])

In [9]:
#TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, stop_words = 'english')
tfidf_train_2 = tv.fit_transform(trainData);
tv2 = TfidfVectorizer(vocabulary = tv.vocabulary_)
tfidf_test_2 = tv2.fit_transform(testData)
print("the shape of train is "+repr(tfidf_train_2.shape))
print("the shape of test is "+repr(tfidf_test_2.shape))

the shape of train is (60000, 54667)
the shape of test is (10000, 54667)


In [0]:
from sklearn import metrics
def calculate_result(actual,pred):
    m_precision = metrics.precision_score(actual,pred,average='micro')
    m_recall = metrics.recall_score(actual,pred,average='micro')
    print('predict info:')
    print('precision:{0:.3f}'.format(m_precision))
    print('recall:{0:0.3f}'.format(m_recall))
    print('f1-score:{0:.3f}'.format(metrics.f1_score(actual,pred,average='micro')))

In [0]:
import time
def saveCSV(pred, modName):
  pred = pd.DataFrame(pred, columns=['Category'])
  pred.to_csv('predResult.csv', index=True, index_label='Id', header=True)
  fileName = modName+''+time.strftime("%Y%m%d%H", time.localtime())+'.csv'
  uploaded = drive.CreateFile({'title': fileName})
  uploaded.SetContentFile('predResult.csv')
  uploaded.Upload()

In [11]:
from sklearn.naive_bayes import BernoulliNB
current_time = datetime.now()
Bclf = BernoulliNB()
Bclf.fit(tfidf_train_2, trainTar)
varA = Bclf.score(tfidf_test_2, testTar)
print("Training took time ", datetime.now() - current_time)
print(varA)

Training took time  0:00:00.356745
0.5126


In [29]:
from sklearn.naive_bayes import MultinomialNB
#fea_test = vectorizer.fit_transform(newsgroups_test.data)
#create the Multinomial Naive Bayesian Classifier

current_time = datetime.now()
clf = MultinomialNB(alpha = 0.32)  #0.32
clf.fit(tfidf_train_2, trainTar)
#pred = clf.predict(tfidf_test_2)
#calculate_result(testTar,pred)
varA = clf.score(tfidf_test_2, testTar)
#saveCSV(pred, 'MultNB')
print("Training took time ", datetime.now() - current_time)
print(varA)

Training took time  0:00:00.317397
0.5772


In [34]:
# with cross-validation
current_time = datetime.now()

Cscore = cross_val_score(clf, tfidf_train_2, trainTar, cv=20)

print("Training took time ", datetime.now() - current_time)
print(Cscore)
print("Accuracy: %0.3f (+/- %0.2f)" % (Cscore.mean(), Cscore.std() * 2))

Training took time  0:00:06.614722
[0.57693586 0.56729811 0.5525266  0.57646277 0.58064516 0.57903494
 0.57404326 0.56472546 0.58441558 0.55814728 0.56466667 0.5730487
 0.57223891 0.56527546 0.56260434 0.57434013 0.55967904 0.56603143
 0.57525084 0.55351171]
Accuracy: 0.569 (+/- 0.02)


In [12]:
from sklearn.svm import SVC

current_time = datetime.now()
svclf = SVC(kernel = 'linear')#default with 'rbf' linear is best now
svclf.fit(tfidf_train_2, trainTar)
svclf.score(tfidf_test_2, testTar)
#pred = svclf.predict(tfidf_test_2)
#calculate_result(testTar,pred)
Cscore = cross_val_score(svclf, tfidf_test_2, testTar, cv=3)
print("Training took time ", datetime.now() - current_time)
print(Cscore)
print("Accuracy: %0.3f (+/- %0.2f)" % (Cscore.mean(), Cscore.std() * 2))

Training took time  0:13:06.074546
[0.39383049 0.39243924 0.38972356]
Accuracy: 0.392 (+/- 0.00)


In [0]:
from sklearn import tree

current_time = datetime.now()
clf = tree.DecisionTreeClassifier()
clf = clf.fit(tfidf_train_2, trainTar)
#pred = clf.predict(tfidf_test_2)
#calculate_result(testTar,pred)

Cscore = cross_val_score(clf, tfidf_train_2, trainTar, cv=3)
print("Training took time ", datetime.now() - current_time)
print(Cscore)
print("Accuracy: %0.3f (+/- %0.2f)" % (Cscore.mean(), Cscore.std() * 2))

predict info:
precision:0.332
recall:0.332
f1-score:0.332


In [13]:
from sklearn.linear_model import LogisticRegression

current_time = datetime.now()

clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial', max_iter=4000).fit(tfidf_train_2, trainTar)
#pred = clf.predict(tfidf_test_2)
#calculate_result(testTar,pred)
clf.score(tfidf_test_2, testTar)
#saveCSV(pred, 'LogReg')

Cscore = cross_val_score(clf, tfidf_train_2, trainTar, cv=4)
print("Training took time ", datetime.now() - current_time)
print(Cscore)
print("Accuracy: %0.3f (+/- %0.2f)" % (Cscore.mean(), Cscore.std() * 2))


Training took time  0:04:34.692869
[0.53661625 0.54336378 0.53707161 0.53974923]
Accuracy: 0.539 (+/- 0.01)


In [0]:
from sklearn.neighbors import KNeighborsClassifier
knnclf = KNeighborsClassifier(n_neighbors=20)#default with k=5
knnclf.fit(tfidf_train_2, trainTar)
pred = knnclf.predict(tfidf_test_2)
calculate_result(testTar,pred)

predict info:
precision:0.067
recall:0.067
f1-score:0.067


In [0]:
# from sklearn.naive_bayes import GaussianNB
# gnb = GaussianNB()
# pred = gnb.fit(tfidf_train_2.toarray(), trainTar).predict(tfidf_test_2.toarray())
# calculate_result(testTar,pred)